In [17]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = '../data/Pandora_tiny_restructured'
validation_data_dir = '../data/Pandora_tiny_restructured'
nb_train_samples = 224
nb_validation_samples = 224
epochs = 50
batch_size = 16


In [18]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, nb_train_samples // batch_size)
np.save(open('bottleneck_features_train.npy', 'w'),
        bottleneck_features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, nb_validation_samples // batch_size)
np.save(open('bottleneck_features_validation.npy', 'w'),
        bottleneck_features_validation)

Found 224 images belonging to 3 classes.
Found 224 images belonging to 3 classes.


In [19]:
from keras.utils.np_utils import to_categorical

train_data = np.load(open('bottleneck_features_train.npy'))
train_labels = np.array([0]*65 + [1]*14 + [2]*145)
train_labels = to_categorical(train_labels, num_classes=3)

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0]*65 + [1]*14 + [2]*145)
validation_labels = to_categorical(validation_labels, num_classes=3)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path,overwrite=True)

Train on 224 samples, validate on 224 samples
Epoch 1/50
224/224 [==============================] - 0s - loss: 2.8238 - acc: 0.6250 - val_loss: 2.1077 - val_acc: 0.6786
Epoch 2/50
224/224 [==============================] - 0s - loss: 1.2790 - acc: 0.8438 - val_loss: 0.4448 - val_acc: 0.9062
Epoch 3/50
224/224 [==============================] - 0s - loss: 0.7753 - acc: 0.7991 - val_loss: 0.4014 - val_acc: 0.9107
Epoch 4/50
224/224 [==============================] - 0s - loss: 0.4759 - acc: 0.8571 - val_loss: 0.1825 - val_acc: 0.9196
Epoch 5/50
224/224 [==============================] - 0s - loss: 0.3475 - acc: 0.8973 - val_loss: 0.4304 - val_acc: 0.8393
Epoch 6/50
224/224 [==============================] - 0s - loss: 0.2307 - acc: 0.9330 - val_loss: 0.1236 - val_acc: 0.9420
Epoch 7/50
224/224 [==============================] - 0s - loss: 0.1894 - acc: 0.9420 - val_loss: 0.1410 - val_acc: 0.9330
Epoch 8/50
224/224 [==============================] - 0s - loss: 0.1508 - acc: 0.9241 - val_l